#### Edmond-Karp Algorithm for Max-Flow 

In [24]:
import numpy as np

#### Edmond-Karp Algorithm implementation

In [25]:
def max_flow(adjacency_matrix, verbose=False):
    # initialize zero flow
    flow = np.zeros_like(adjacency_matrix)
    # initialize label tuples
    labels = [0]*len(adjacency_matrix)
    # intialize empty BFS queue
    Q = []
    # add source to the queue
    Q.append(0)
    labels[0] = (float('inf'), None, None)
    num_iterations = 0
    
    # perform iterations of label-enhanced BFS until the queue is empty
    while len(Q) > 0:
        # pop the first element from the queue
        i = Q.pop(0)
        if verbose: print(f"Front vertex in queue: {i}, current label: {labels[i]}")
        l_i, _, _ = labels[i]
        # visit all forward edges from the current node
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i,j] > 0:
                # check if j is unlabeled
                if labels[j] == 0:
                    # check if edge (i,j) has unused capacity
                    r_ij = adjacency_matrix[i,j] - flow[i,j]
                    if r_ij > 0:
                        # add j to the queue
                        Q.append(j)
                        # label j
                        l_j = min(l_i, r_ij)
                        labels[j] = (l_j, i, '+')
                        if verbose: print(f"Labeled vertex: {j}, label: {labels[j]}")

        # visit all backward edges
        for j in range(adjacency_matrix.shape[0]):
            if adjacency_matrix[j,i] > 0:
                # check if j is unlabeled
                if labels[j] == 0:
                    # check if edge (j,i) has unused capacity
                    x_ji = flow[j,i]
                    if x_ji > 0:
                        # add j to the queue
                        Q.append(j)     
                        # label j
                        l_j = min(l_i, x_ji)
                        labels[j] = (l_j, i, '-')
                        if verbose: print(f"Labeled vertex: {j}, label: {labels[j]}")

        # if sink is labeled, augment the flow, reset the labels and put the source back to the queue
        if labels[-1] != 0:
            l_n, i, direction = labels[-1]
            j = len(labels)-1
            while j != 0:
                l_j, i, direction = labels[j]
                if direction == '+':
                    flow[i,j] += l_n
                else:
                    flow[j,i] -= l_n    
                j = i     
            # erase the labels
            labels = [0]*len(adjacency_matrix)
            # reset the queue and add source vertex
            Q = [0] 
            labels[0] = (float('inf'), None, None)     
            num_iterations += 1      
            if verbose: print("Flow augmented!")                        

    # find the min-cut: C(X, X_c), the labaled vertices are X and the unlabeled vertices are X_c 
    min_cut = set()
    labeled_vertices = []
    unlabeled_vertices = []
    for i in range(len(labels)):
        if labels[i] != 0:
            labeled_vertices.append(i)
        else:
            unlabeled_vertices.append(i)    
    for i in labeled_vertices:
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i,j] > 0 and j not in labeled_vertices:
                min_cut.add((i,j))

    if verbose: print(f"Max flow value: {np.sum(flow[0,:])}, min-cut: {min_cut}, number of iterations: {num_iterations}")

    max_flow_value = np.sum(flow[0,:])
    return max_flow_value, flow, min_cut


Create an adjacency matrix for an example network.

<img src="example_1.png" width="450" height="250">



In [16]:
network_adj = np.array([[0,2,0,3,0,0],
                        [0,0,5,0,3,0],
                        [0,0,0,0,0,2],
                        [0,0,1,0,0,0],
                        [0,0,0,0,0,4],
                        [0,0,0,0,0,0]])            

Now let's test our algorithm implementation on this example network

In [27]:
max_flow_value, flow, min_cut = max_flow(network_adj, verbose=True)

Front vertex in queue: 0, current label: (inf, None, None)
Labeled vertex: 1, label: (2, 0, '+')
Labeled vertex: 3, label: (3, 0, '+')
Front vertex in queue: 1, current label: (2, 0, '+')
Labeled vertex: 2, label: (2, 1, '+')
Labeled vertex: 4, label: (2, 1, '+')
Front vertex in queue: 3, current label: (3, 0, '+')
Front vertex in queue: 2, current label: (2, 1, '+')
Labeled vertex: 5, label: (2, 2, '+')
Flow augmented!
Front vertex in queue: 0, current label: (inf, None, None)
Labeled vertex: 3, label: (3, 0, '+')
Front vertex in queue: 3, current label: (3, 0, '+')
Labeled vertex: 2, label: (1, 3, '+')
Front vertex in queue: 2, current label: (1, 3, '+')
Labeled vertex: 1, label: (1, 2, '-')
Front vertex in queue: 1, current label: (1, 2, '-')
Labeled vertex: 4, label: (1, 1, '+')
Front vertex in queue: 4, current label: (1, 1, '+')
Labeled vertex: 5, label: (1, 4, '+')
Flow augmented!
Front vertex in queue: 0, current label: (inf, None, None)
Labeled vertex: 3, label: (2, 0, '+')
Fr